# Schwinger bosons for two coupled spins

In [ ]:
using KadanoffBaym
using LinearAlgebra
using NumericalIntegration
using Parameters
# using Plots

using PyPlot
using PyCall
using ProgressMeter
qt = pyimport("qutip")

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=16)

### Hamiltonian

\begin{align}
    H &=  h \sum_{i=1, 2} \left(a^\dagger_{i} a^{\phantom{\dagger}}_{i} - b^\dagger_{i} b^{\phantom{\dagger}}_{i} \right) + J \left( a^\dagger_{1}b^{\phantom{\dagger}}_{1} b^\dagger_{2} a^{\phantom{\dagger}}_{2} + \mathrm{h.c.}\right) 
\end{align}

https://journals.aps.org/prb/abstract/10.1103/PhysRevB.98.224304

## KadanoffBaym.jl

In [ ]:
# NOTE: not type-stable
NumericalIntegration._zero(x, y::Vector{<:AbstractMatrix}) = zero(first(y))

# Container for problem data
struct ProblemData
  GL::GreenFunction{ComplexF64, Array{ComplexF64,4}, Lesser}
  GG::GreenFunction{ComplexF64, Array{ComplexF64,4}, Greater}
  ΣL::GreenFunction{ComplexF64, Array{ComplexF64,4}, Lesser}
  ΣG::GreenFunction{ComplexF64, Array{ComplexF64,4}, Greater}
  h::Matrix{ComplexF64}
  J::Float64
  
  # Initialize problem
  function ProblemData(GL0::Matrix{ComplexF64}, h::Matrix{ComplexF64}, J::Float64)
    @assert h == h' "A complex Hamiltonian requires revision of the equations"
    
    data = new(
      GreenFunction(reshape(GL0, size(GL0)...,1,1), Lesser),
      GreenFunction(reshape(GL0 - 1.0im * I, size(GL0)...,1,1), Greater),
      GreenFunction(zeros(ComplexF64, size(GL0)...,1,1), Lesser),
      GreenFunction(zeros(ComplexF64, size(GL0)...,1,1), Greater),
      h,
      J
    )
    
    # Initialize self-energies
    calculate_Σ!(data, 1, 1)
    
    return data
  end
end

# Vertical rhs
function rhs_vert(data, times, t1, t2)
  @unpack GL,GG,ΣL,ΣG,h = data
  
#   @unpack GL,GG,#=ΣL,ΣG,=#h,J = data
#   is = [[2, 1, 4, 3], [3, 4, 1, 2], [4, 3, 2, 1]]  
#   ΣL(t1,t2) = -J^2 * GL[t1,t2][is[1],is[1]] .* GL[t1,t2][is[2],is[2]] .* GG[t2,t1][is[3],is[3]] |> (diagm ∘ diag)
#   ΣG(t1,t2) = -J^2 * GG[t1,t2][is[1],is[1]] .* GG[t1,t2][is[2],is[2]] .* GL[t2,t1][is[3],is[3]] |> (diagm ∘ diag)
#   ∫dt(i,j,A,B) = sign(j-i) * integrate(times[min(i,j):max(i,j)], 
#     [A(t1,t) .* B[t,t2] for t=min(i,j):max(i,j)], TrapezoidalFast())
  
  # real-time collision integral
  ∫dt(i,j,A,B) = sign(j-i) * integrate(times[min(i,j):max(i,j)], 
    [A[t1,t] .* B[t,t2] for t=min(i,j):max(i,j)], TrapezoidalFast())

  dGL = -1.0im * (h * GL[t1,t2] + ∫dt(1,t1,ΣG,GL) - ∫dt(1,t1,ΣL,GL) + ∫dt(1,t2,ΣL,GL) - ∫dt(1,t2,ΣL,GG))
  dGG = -1.0im * (h * GG[t1,t2] + ∫dt(1,t1,ΣG,GG) - ∫dt(1,t1,ΣL,GG) + ∫dt(1,t2,ΣG,GL) - ∫dt(1,t2,ΣG,GG))
  return [dGL, dGG] .|> (diagm ∘ diag)
end

# Diagonal rhs
function rhs_diag(data, times, t1)
  @unpack GL,GG,h,ΣL,ΣG = data
#   @unpack GL,GG,h#=,ΣL,ΣG=# = data
#   is = [[2, 1, 4, 3], [3, 4, 1, 2], [4, 3, 2, 1]]
#   ΣL(t1,t2) = -J^2 * GL[t1,t2][is[1],is[1]] .* GL[t1,t2][is[2],is[2]] .* GG[t2,t1][is[3],is[3]] |> (diagm ∘ diag)
#   ΣG(t1,t2) = -J^2 * GG[t1,t2][is[1],is[1]] .* GG[t1,t2][is[2],is[2]] .* GL[t2,t1][is[3],is[3]] |> (diagm ∘ diag)
#   ∫dt(A::Function,B::GreenFunction) = integrate(times, [A(t1,t) .* B[t,t1] for t=1:t1], TrapezoidalFast())
#   ∫dt(A::GreenFunction,B::Function) = integrate(times, [A[t1,t] .* B(t,t1) for t=1:t1], TrapezoidalFast())
  
  # commutator
  ⋊(a,b) = a * b - b * a
  
  # real-time collision integral
  ∫dt(A,B) = integrate(times, [A[t1,t] .* B[t,t1] for t=1:t1], TrapezoidalFast())

  I = ∫dt(ΣG,GL) - ∫dt(ΣL,GG) + ∫dt(GL,ΣG) - ∫dt(GG,ΣL)
  
  dGL = -1.0im * (h ⋊ GL[t1,t1] + I)
  dGG = -1.0im * (h ⋊ GG[t1,t1] + I)
  return [dGL, dGG] .|> (diagm ∘ diag)
end

# Self-energy
function calculate_Σ!(data, t1, t2)
  @unpack GL,GG,ΣL,ΣG,h,J = data
  
  if (n = size(GL,3)) > size(ΣL,3)
    resize!(ΣL, n)
    resize!(ΣG, n)
  end
  
  is = [[2, 1, 4, 3], [3, 4, 1, 2], [4, 3, 2, 1]]
  
  ΣL[t1,t2] = -J^2 * GL[t1,t2][is[1],is[1]] .* GL[t1,t2][is[2],is[2]] .* GG[t2,t1][is[3],is[3]] |> (diagm ∘ diag)
  ΣG[t1,t2] = -J^2 * GG[t1,t2][is[1],is[1]] .* GG[t1,t2][is[2],is[2]] .* GL[t2,t1][is[3],is[3]] |> (diagm ∘ diag)
end

# Integration boundaries
tspan = (0.0, 2.0)

# Problem data
data = begin

  # Parameters
  h = -1.0
  H = ComplexF64[-h 0 0 0; 0 -h 0 0; 0 0 h 0; 0 0 0 h];
  J = 0.05

  # Initial condition
  GL0 = -1.0im * [1 0 0 0; 0 0 0 0; 0 0 0 0; 0 0 0 1]
  
  ProblemData(GL0, H, J)
end

# Integration
state, _ = kbsolve(
  (u, x...) -> rhs_vert(data, x...), 
  (u, x...) -> (println(" t: $(x[1][x[2]])"); rhs_diag(data, x...)), 
  [data.GL, data.GG], 
  tspan[1], 
  tspan[2]; 
  update_time! = (_, x...) -> calculate_Σ!(data, x...), 
  atol=1e-8, 
  rtol=1e-6, 
  init_dt=1e-5,
  max_dt=2e-2)

In [ ]:
# p1 = plot(state.t, -imag(diag(data.GL.data[1,1,:,:])), label="J = $J")
# p1 = plot!(state.t, -imag(diag(data.GL.data[2,2,:,:])), label="J = $J")
# ylims!(0.99, 1.0)

# p2 = plot(state.t, [-imag(sum(diag(data.GL.data[:,:,t,t]))) for t in eachindex(state.t)])

# display(p1)
# display(p2)

## QuTiP benchmark

In [ ]:
# time parameters
n = length(state.t) - 1
stop = Int(n/2) + 1;
# delta_t = T/n

times = range(tspan[1]; stop=tspan[2], length=length(state.t));

In [ ]:
n_max = 2; # Fock-space truncation

psi0_list = [qt.basis(n_max + 1, 1),
             qt.basis(n_max + 1, 0),
             qt.basis(n_max + 1, 0),
             qt.basis(n_max + 1, 1)]

psi0 = qt.tensor(psi0_list);

# operators

id_list = [qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1)]

dg1_list = [qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1)] # cp.deepcopy(id_list)
dg1_list[1] = qt.destroy(n_max + 1)
dg1 = qt.tensor(dg1_list)

dg2_list = [qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1)] # cp.deepcopy(id_list)
dg2_list[2] = qt.destroy(n_max + 1)
dg2 = qt.tensor(dg2_list)

de1_list = [qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1)] # cp.deepcopy(id_list)
de1_list[3] = qt.destroy(n_max + 1)
de1 = qt.tensor(de1_list)

de2_list = [qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1)] # cp.deepcopy(id_list)
de2_list[4] = qt.destroy(n_max + 1)
de2 = qt.tensor(de2_list);

# Hamiltonian

H = -h * dg1.dag() * dg1
H += -h * dg2.dag() * dg2
H += +h * de1.dag() * de1
H += +h * de2.dag() * de2
H += J * (de1.dag() * dg1 * dg2.dag() * de2 + dg1.dag() * de1 * de2.dag() * dg2)

# observables
obs = [dg1.dag() * dg1, dg2.dag() * dg2, de1.dag() * de1, de2.dag() * de2,
       dg1.dag() * de1, dg2.dag() * de2, dg2.dag() * de1, de2.dag() * dg1,
       dg1 * dg1.dag()];

# quickly solve once for observables
me = qt.mesolve(H, psi0, times, [], obs);

# solve for the time-dependent density matrix
t_sols = qt.mesolve(H, psi0, times); # t_sols.states returns state vectors

In [ ]:
tau_t_sols_11 = Dict()
tau_t_sols_22 = Dict()
@showprogress 1 for k in 1:n + 1
    tau_t_sols_11[k] = qt.mesolve(H, dg1 * t_sols.states[k] * t_sols.states[k].dag(), times).states
    tau_t_sols_22[k] = qt.mesolve(H, dg2 * t_sols.states[k] * t_sols.states[k].dag(), times).states
#     tau_t_sols_11[k] = qt.mesolve(H, a_1 * t_sols.states[k], times).states
#     tau_t_sols_22[k] = qt.mesolve(H, a_2 * t_sols.states[k], times).states    
end

dg1_dag_dg1 = zeros(ComplexF64, n + 1, n + 1)
dg2_dag_dg2 = zeros(ComplexF64, n + 1, n + 1)
@showprogress 1 for k in 1:(n + 1)
    for l in 1:(n + 1)
        dg1_dag_dg1[k, l] = (dg1.dag() * tau_t_sols_11[k][l]).tr()    
        dg2_dag_dg2[k, l] = (dg2.dag() * tau_t_sols_22[k][l]).tr()   
    end
end

In [ ]:
# reshape the above array to fit into our two-time "matrix" structure 
# see the plot below for illustration
unskewed_dg1_dag_dg1 = zeros(ComplexF64, n + 1, 2*(n + 1) - 1)
unskewed_dg2_dag_dg2 = zeros(ComplexF64, n + 1, 2*(n + 1) - 1)
for (k, x) in enumerate([dg1_dag_dg1[k, :] for k in 1:(n + 1)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_dg1_dag_dg1[k, ind] = y 
    end
end

for (k, x) in enumerate([dg2_dag_dg2[k, :] for k in 1:(n + 1)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_dg2_dag_dg2[k, ind] = y  
    end
end

In [ ]:
figure(figsize=(6, 2))
subplot(121)
imshow(real(dg1_dag_dg1), cmap="plasma")

subplot(122)
imshow(real(unskewed_dg1_dag_dg1), cmap="plasma")

tight_layout()

## Plotting

In [ ]:
# Interpolation
interpolate = pyimport("scipy.interpolate")

const interp2d = interpolate.interp2d
# const rectSpline = interpolate.RectBivariateSpline

# Data
GL11 = [data.GL[1,1,t1,t2] for t1 ∈ eachindex(state.t), t2 ∈ eachindex(state.t)]

# Grid
time_grid = hcat([state.t for _ in eachindex(state.t)]...)
tX = vcat([eachcol(time_grid)...]...)
tY = vcat([eachrow(time_grid)...]...)

GL_re_itp = interp2d(tX, tY, vcat([eachrow(real(GL11))...]...))
GL_im_itp = interp2d(tX, tY, vcat([eachrow(imag(GL11))...]...))
GL_ip(t1,t2) = GL_re_itp(t1,t2) + 1.0im * GL_im_itp(t1,t2)

In [ ]:
import Interpolations
GL11 = [data.GL[1,1,t1,t2] for t1 ∈ eachindex(state.t), t2 ∈ eachindex(state.t)]
GL_ipp = Interpolations.interpolate((state.t, state.t), GL11, Gridded(Linear()))

In [ ]:
function equal_time_values(GX::Array{Array{ComplexF64, 2}, 2})::Array{Array{ComplexF64, 1}, 2}
    dim = size(GX)[1]
    Y = [zeros(ComplexF64, n + 1) for _ in 1:dim, _ in 1:dim]
    for b in 1:dim, a in 1:dim
        Y[a, b] = [GX[a, b][k, k] for k in 1:n+1]
    end
    
    return Y
end

function full_tau_array(X::Array{ComplexF64, 2})::Array{ComplexF64, 1}
    Y = [X[stop - (k - 1), stop + (k - 1)] for k in 1:stop]
    Y_reversed = [Y[stop - (k - 1)] for k in 1:stop]
    return vcat(-conj(Y_reversed[1:end-1]), Y)    
end

# function difference_time_values(GX)
#     dim = size(GX, 1)
#     Y_full = [zeros(ComplexF64, n + 1) for _ in 1:dim, _ in 1:dim]
#     for b in 1:dim, a in 1:dim
#         Y_full[a, b] = full_tau_array(GX[a, b])
#     end 
    
#     return Y_full
# end

dg1_dag_dg1_tau = imag(full_tau_array(1.0im .* unskewed_dg1_dag_dg1))
dg2_dag_dg2_tau = imag(full_tau_array(1.0im .* unskewed_dg2_dag_dg2));

# defining the tau times
times_tau = 2 .* vcat([-times[stop - (k - 1)] for k in 1:stop - 1], times[1:stop]);

In [ ]:
GL_tau = full_tau_array(GL_ip(times,times));
# GL_tau, _ = wigner_transform(GL_ip(times,times); ts=times, fourier=false)

In [ ]:
plot(times_tau, -imag(GL_tau), lw=3)
# plot(times_tau, -imag(GL_tau[2, 2]), c="b")
plot!(times_tau, dg1_dag_dg1_tau, lw=3, alpha=0.5)
# plot(times_tau, dg2_dag_dg2_tau, "--", c="k", lw=3, alpha=0.5)
# ax.set_xlim(-T, T)  
# ax.set_ylim(None, 0.01)
# ax.set_xlabel("\$\\tau\$") 

In [ ]:
GL_1 = -GL_ip(times,times) |> diag |> imag

fig, ax = plt.subplots(2, 2, figsize=(12, 6))

ax = plt.subplot(2, 2, 1)
ax.plot(state.t, data.GL.data[1,1,:,:] |> ((-) ∘ imag ∘ diag))
ax.plot(times, me.expect[1], "--", c="k", lw=3, alpha=0.5)
ax.set_xlabel("\$T\$") 
ax.set_xlim(0, state.t[end]) 

ax = plt.subplot(2, 2, 2)
ax.plot(state.t, data.GL.data[2,2,:,:] |> ((-) ∘ imag ∘ diag))
ax.plot(times, me.expect[2], "--", c="k", lw=3, alpha=0.5)
ax.set_xlabel("\$T\$") 
ax.set_xlim(0, state.t[end]) 

ax = plt.subplot(2, 2, 3)
ax.plot(state.t, data.GL.data[3,3,:,:] |> ((-) ∘ imag ∘ diag))
ax.plot(times, me.expect[3], "--", c="k", lw=3, alpha=0.5)
ax.set_xlabel("\$T\$") 
ax.set_xlim(0, state.t[end]) 

ax = plt.subplot(2, 2, 4)
plot(state.t, [data.GL.data[1,1,1,k] |> ((-) ∘ imag) for k in 1:n+1])
plot(times, [unskewed_dg1_dag_dg1[1, k] for k in 1:n+1] |> real, "--", c="k", lw=3, alpha=0.5)
ax.set_xlabel("\$t'\$") 
ax.set_xlim(0, state.t[end]) 
tight_layout()

In [ ]:
GL_1 = -GL_ipp(times,times) |> diag |> imag

plot(state.t, data.GL.data[1,1,:,:] |> diag |> imag |> (-))
plot!(times, GL_1)